# Process Model for Machine Learning in Digital Health

<img src="assets/process_model.png" width="40%"/>

## 1. Requirements Analysis

## 2. Data Acquisition

## 3. Data Preparation

## 4. Predictive Modelling

## 5. Evaluation

## 6. Deployment